In [25]:
from transformers import AutoTokenizer, LongformerForSequenceClassification
from tqdm import tqdm
import torch
from sklearn.metrics import confusion_matrix
import pandas as pd

In [22]:
# download the model and tokenizer from huggingface and replace this with your own path
tokenizer = AutoTokenizer.from_pretrained('/home/ubuntu/transaction/data/tokenizer')
model = LongformerForSequenceClassification.from_pretrained('/home/ubuntu/transaction/finetune3_results/checkpoint-2400')
config = model.config

In [23]:
# load the example dataset
dataset = pd.read_csv('example.csv')
label, data = [], []
for index, row in tqdm(dataset.iterrows(), desc="Processing rows", total=len(dataset)):
    label_row, data_row, transaction_hash_row = row['label'], row['data'], row['transaction_hash']
    label.append(label_row)
    data.append(data_row)

Processing rows: 100%|██████████| 10/10 [00:00<00:00, 13391.78it/s]


In [24]:
# make predictions
predictions = []
for i in range(len(data)):
    inputs = tokenizer(data[i], return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predictions.append(model.config.id2label[predicted_class_id])

# print the confusion matrix
print(confusion_matrix(label, predictions))

[[2 0 0 0 0]
 [0 2 0 0 0]
 [0 0 2 0 0]
 [0 0 0 2 0]
 [0 0 0 0 2]]
